In [ ]:
import os
import pandas as pd

#請更改下載位置!!
folder = r"C:\Users\techang\Desktop\wof\60\2024"

# 抓資料夾底下所有 Excel 檔
files = [f for f in os.listdir(folder) if f.endswith(".xlsx") or f.endswith(".xls")]

all_data = []

for f in files:
    file_path = os.path.join(folder, f)
    try:
        if f.endswith(".xlsx"):
            df = pd.read_excel(file_path, engine="openpyxl")  # 新格式
        else:
            df = pd.read_excel(file_path, engine="xlrd")       # 舊格式

        # 記錄這筆資料來自哪個檔案
        df["source_file"] = f

        print(f"{f} 讀到 {len(df)} 筆")
        all_data.append(df)

    except Exception as e:
        print(f"❌ 讀取失敗：{f}；原因：{e}")

# 合併全部資料
combined = pd.concat(all_data, ignore_index=True)

print("\n========================")
print(f"合併後總筆數：{len(combined)}")
print("========================")

# 👉 想檢查「內容完全一樣」，就不要把 source_file 算進去
cols_for_check = [c for c in combined.columns if c != "source_file"]

# 找出「整列完全相同」的資料（keep=False 是把所有重複列都抓出來）
dup_rows = combined[combined.duplicated(subset=cols_for_check, keep=False)]

print("完全重複的列數：", len(dup_rows))

if len(dup_rows) > 0:
    print("\n⚠️ 有發現重複資料，前幾列如下：")
    print(dup_rows.head(20))   # 可以改成 head(100) 看多一點

    # 也可以輸出成一個檔案方便開 Excel 看
    dup_rows.to_excel(os.path.join(folder, "重複資料列表.xlsx"), index=False)
    print("\n已輸出『重複資料列表.xlsx』到資料夾。")
else:
    print("✅ 沒有任何整列完全一樣的重複資料。")